# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [20]:
!cat publications.csv




2017-03-01,Team-based data management instruction at small liberal arts colleges,IFLA Journal,"This paper describes a collaborative approach taken by librarians at five small, regional liberal arts colleges to developing/enhancing research data management services on their campuses. The five colleges collectively belong to a consortium known as the Northwest Five Consortium. Over 10 months, librarians from the five schools collaborated to plan a data management and curation workshop with the goals of developing relationships with researchers working with data, developing their own research data management skills and services, and building a model for future training and outreach around institutional research data management services. This workshop brought together research teams including faculty, students, and librarians, and incorporated active learning modules as well as in-depth pre-workshop discussion. This article will discuss the context and background for this workshop, the 

## Import pandas

We are using the very handy pandas library for dataframes.

In [13]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [14]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2015,Walking a Mile in the User's Shoes: Customer J...,Internet Reference Services Quarterly,This article introduces using Customer Journey...,"Marquez, J. J., Downey, A., & Clement, R. (201...",paper-2016-walking-a-mile,https://doi.org/10.1080/10875301.2015.1107000
1,2016,The Data Librarian in the Liberal Arts College,Databrarianship: The Academic Data Librarian i...,A chapter from <i>Databrarianship: The Academi...,"Clement, R. (2016). The Data Librarian in the ...",chapter-data-librarian-LAC,https://repository.middlebury.edu/islandora/ob...
2,2017,Team-based data management instruction at smal...,IFLA Journal,This paper describes a collaborative approach ...,"Clement, R., Blau, A., Abbaspour, P., & Gandou...",paper-team-based-data-management-instruction,http://doi.org/10.1177/0340035216678239


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [15]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [16]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

TypeError: 'int' object is not subscriptable

These files are in the publications directory, one directory below where we're working from.

In [17]:
!ls ../_publications/

15-paper-2016-walking-a-mile.md
16-chapter-data-librarian-LAC.md
17-paper-team-based-data-management-instruction.md


In [18]:
!cat ../_publications/15-paper-2016-walking-a-mile.md

---
title: "Walking a Mile in the User's Shoes: Customer Journey Mapping as a Method to Understanding the User Experience"
collection: publications
permalink: /publication/10/2/15-paper-2016-walking-a-mile
excerpt: 'This article introduces using Customer Journey Maps (CJM) in libraries and the role mapping can play in visualizing the user&apos;s journey in order to help library staff better understand and optimize the user&apos;s experience. The purpose of this article is to demonstrate the importance and relevance of the mapping process for any library user experience. The article will also review findings from the Reed College Library use of mapping discovered during our own review of services and resource usage.'
date: 10/2/15
venue: 'Internet Reference Services Quarterly'
paperurl: 'https://doi.org/10.1080/10875301.2015.1107000'
citation: 'Marquez, J. J., Downey, A., &amp; Clement, R. (2015). Walking a Mile in the User’s Shoes: Customer Journey Mapping as a Method to Understanding 